<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [2]:
# 모듈 import
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException


from bs4 import BeautifulSoup
import time 

In [3]:
target_categories = ['게요리',
 '곱창,막창,양',
 '과일,주스전문점',
 '국밥',
 '김밥',
 '낙지요리',
 '다이어트,샐러드',
 '닭갈비',
 '닭발',
 '닭볶음탕',
 '닭요리',
 '대게요리',
 '덮밥',
 '도시락,컵밥',
 '돈가스',
 '돼지고기구이',
 '떡볶이', '라면'
 '마라탕',
 '만두',                
 '맥주,호프',
 '멕시코,남미음식',
 '바(BAR)',
 '백반,가정식',
 '베이커리',
 '베트남음식',
 '분식',
 '빙수',
 '복어요리',                     
 '생선회',
 '샤브샤브',
 '소고기구이',
 '순대,순댓국',
 '술집',
 '스테이크,립',
 '스파게티,파스타전문',
 '아시아음식',
 '양꼬치',
 '양식',
 '오므라이스',
 '요리주점',
 '육류,고기요리',
 '이자카야',
 '이탈리아음식',
 '인도음식',
 '일본식라면',
 '일식당',
 '일식튀김,꼬치',
 '족발,보쌈',
 '주꾸미요리',
 '중식당',
 '찜닭',
 '초밥,롤',
 '치킨,닭강정',
 '카레',
 '카페',
 '카페,디저트',
 '칼국수,만두',
 '태국음식',
 '프랑스음식',
 '피자',
 '한식',
 '해물,생선요리',
 '햄버거',
 '향토음식',
 '홍차전문점']

In [5]:
# 각자 동에 맞는 거로!

restaurant = pd.read_excel('./서울시마포구일반음식점인허가정보.xls') 
restaurant = restaurant[restaurant['영업상태명'] == '영업/정상']

# NaN 값을 빈 문자열로 대체, 안 하면 에러남
restaurant['지번주소'] = restaurant['지번주소'].fillna('')

# '지번주소' 열이 '서울특별시 서대문구 창천동'으로 시작하는 행만 선택
target_dong = '서울특별시 마포구 염리동'
dong = restaurant[restaurant['지번주소'].str.startswith(target_dong)]

# dong에서 도로명주소를 기준으로 검색할 것이기 때문에, 주소 기준 중복인 행을 다 없앰

dong = dong.drop_duplicates(subset='도로명주소', keep=False)
dong.reset_index(drop=True, inplace=True)
dong

C:\Users\uje10\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
0,3130000,3130000-101-1976-00926,19760901,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,14.30,NaN,NaN,NaN
1,3130000,3130000-101-1981-02292,1981-07-13,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3130000,3130000-101-1983-00928,19831217,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3130000,3130000-101-1983-02294,19831114,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Y,139.49,NaN,NaN,NaN
4,3130000,3130000-101-1983-04236,19830517,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,N,56.18,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,3130000,3130000-101-2024-00041,2024-01-11,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
134,3130000,3130000-101-2024-00181,2024-03-04,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
135,3130000,3130000-101-2024-00267,2024-03-22,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
136,3130000,3130000-101-2024-00437,2024-05-22,NaN,1,영업/정상,1,영업,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# webdriver 실행
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 검색 결과가 표시되는 iframe으로 전환
def search_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        driver.switch_to.frame("searchIframe")  # 검색 결과 프레임으로 전환
    except NoSuchElementException:
        print("searchIframe을 찾을 수 없습니다.")
        return False
    return True

# 개별 업체의 세부 정보가 표시되는 iframe으로 전환
def entry_iframe():
    driver.switch_to.default_content()  # 차상위 (기본) 프레임으로 전환
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        driver.switch_to.frame("entryIframe")  # 세부 정보 프레임으로 전환
    except NoSuchElementException:
        print("entryIframe을 찾을 수 없습니다.")
        return False
    return True

# type1의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type1():
    search_iframe()  # search 프레임으로 전환
    elems = driver.find_elements(By.CLASS_NAME, 'place_bluelink.C6RjW')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'YwYLL').text
            category = e.find_element(By.CLASS_NAME, 'YzBgS').text
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# type2의 화면에 표시된 업체명을 확인하는 함수
def chk_names_type2(processed_names):
    elems = driver.find_elements(By.CLASS_NAME, 'search_box')
    name_list = []
    category_list = []
    elements = []
    
    for e in elems:
        try:
            name = e.find_element(By.CLASS_NAME, 'search_title').text
            category = e.find_element(By.CLASS_NAME, 'category').text
            
            # 이미 처리된 이름은 패스
            if name in processed_names:
                continue
            
            # 업종이 target_categories에 포함되지 않으면 패스
            if category not in target_categories:
                continue
                
            name_list.append(name)
            category_list.append(category)
            elements.append(e)
        except:
            continue
            
    return elements, name_list, category_list

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type1)
def chk_details():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    search_iframe()  # 다시 검색 결과의 iframe으로 전환
    return url, ID

# 세부 페이지에서 URL과 ID를 추출하는 함수 (type2)
def chk_details_type2():
    if not entry_iframe():  # 세부 정보의 iframe으로 전환
        return float('nan'), float('nan')
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    try:
        # 특정 meta 태그를 찾기
        meta_tag = soup.find('meta', {'id': 'og:url'})

        # URL에서 숫자 부분만 추출
        if meta_tag:
            url = meta_tag['content']
            ID = url.split('/')[4]
    except:
        url = float('nan')
        ID = float('nan')

    return url, ID

# 크롤링 메인 함수
def crawling_main(type):
    global naver_res  # 전역변수 naver_res 사용 (크롤링 데이터를 저장하는 데이터프레임)
    global target_dong
    
    processed_names = set()  # 이미 처리된 업체명을 추적하기 위한 집합
    
    if type == "type1":
        elements, name_list, category_list = chk_names_type1()
        details_func = chk_details
        
        id_list = []
        url_list = []

        for e in elements:
            try:
                e.click()  # 업체명 클릭
                time.sleep(1.5)  # 페이지 로딩 대기
                url, ID = details_func()  # 업체의 세부 정보 가져오기
                url_list.append(url)
                id_list.append(ID)

            except:
                url_list.append(float('nan'))
                id_list.append(float('nan'))
                continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(name_list),
            '도로명주소': [current_keyword] * len(name_list),
            '업체명': name_list,
            '업종': category_list,
            'URL': url_list,
            'ID': id_list
        })
        
        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.reset_index(drop=True, inplace=True)
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장
        
    if type == "type2":
        all_name_list = []
        all_category_list = []
        all_id_list = []
        all_url_list = []
        
        while True:
            elements, name_list, category_list = chk_names_type2(processed_names)
            details_func = chk_details_type2

            if len(elements) == 0:
                break

            for e, name, category in zip(elements, name_list, category_list):
                try:
                    e.click()  # 업체명 클릭
                    time.sleep(1.5)  # 페이지 로딩 대기
                    url, ID = details_func()  # 업체의 세부 정보 가져오기
                    all_url_list.append(url)
                    all_id_list.append(ID)
                    processed_names.add(name)  # 처리된 이름을 추가
                    all_name_list.append(name)
                    all_category_list.append(category)
                    driver.back()  # 뒤로가기 버튼 클릭
                    time.sleep(1.5)  # 뒤로가기 후 페이지 로딩 대기
                except:
                    continue
        
        # 새로운 데이터프레임 생성
        naver_temp = pd.DataFrame({
            '지번주소': [current_jibun] * len(all_name_list),
            '도로명주소': [current_keyword] * len(all_name_list),
            '업체명': all_name_list,
            '업종': all_category_list,
            'URL': all_url_list,
            'ID': all_id_list
        })

        # 기존 데이터프레임과 결합
        naver_res = pd.concat([naver_res, naver_temp])
        naver_res.to_csv(f'./{target_dong.split()[-1]}.csv')  # 결과를 csv 파일로 저장

# 불러올 정보를 담을 빈 데이터프레임 생성
naver_res = pd.DataFrame(columns=['지번주소', '도로명주소', '업체명', '업종', 'URL', 'ID'])

# 소요 시간 측정을 위한 시작 시간 기록
start_time = time.time()

for index, row in dong.iterrows():
    current_jibun = row['지번주소']  # '지번주소'를 저장
    current_keyword = row['도로명주소']  # '도로명주소'를 검색 키워드로 사용
    
    print(f'전체 {len(dong)} 중 {index+1}, 주소:{current_jibun}')
    
    url = f'https://map.naver.com/p/search/{current_keyword}'
    driver.get(url)

    # "sc-1wsjitl dunggE overlap" 클래스가 있는지 확인
    try:
        time.sleep(1.5)  # 페이지 로딩 대기
        overlap_element = driver.find_element(By.CLASS_NAME, 'sc-1wsjitl.dunggE.overlap')
        try:
            more_button = overlap_element.find_element(By.CLASS_NAME, 'link_more')
            more_button.click()
            time.sleep(1.5)  # "더보기" 버튼 클릭 후 페이지 로딩 대기
        except NoSuchElementException:
            pass  # "더보기" 버튼이 없는 경우 그냥 넘어감
        crawling_main("type2")
    except NoSuchElementException:
        try:
            no_result = driver.find_element(By.CLASS_NAME, 'correction_result_text')
            continue
        except:
            try:
                crawling_main("type1")
            except NoSuchElementException:
                continue

    action = ActionChains(driver)

driver.quit()

# 소요 시간 측정을 위한 종료 시간 기록
end_time = time.time()

# 소요 시간 출력
print(f'소요 시간: {end_time - start_time} 초')

전체 138 중 1, 주소:서울특별시 마포구 염리동 148-24번지 
전체 138 중 2, 주소:서울특별시 마포구 염리동 148-13
전체 138 중 3, 주소:서울특별시 마포구 염리동 147-28
전체 138 중 4, 주소:서울특별시 마포구 염리동 172-14 
전체 138 중 5, 주소:서울특별시 마포구 염리동 147-5번지 지상1층, 지하1층 
전체 138 중 6, 주소:서울특별시 마포구 염리동 168-9 재화스퀘어 
전체 138 중 7, 주소:서울특별시 마포구 염리동 148-33번지 
전체 138 중 8, 주소:서울특별시 마포구 염리동 172-15 
전체 138 중 9, 주소:서울특별시 마포구 염리동 173-29 
전체 138 중 10, 주소:서울특별시 마포구 염리동 172-1
전체 138 중 11, 주소:서울특별시 마포구 염리동 9-118번지 1F동 
전체 138 중 12, 주소:서울특별시 마포구 염리동 147-7
전체 138 중 13, 주소:서울특별시 마포구 염리동 178-16번지 
전체 138 중 14, 주소:서울특별시 마포구 염리동 148-20번지 
전체 138 중 15, 주소:서울특별시 마포구 염리동 154-2번지 
전체 138 중 16, 주소:서울특별시 마포구 염리동 149-9번지 
전체 138 중 17, 주소:서울특별시 마포구 염리동 8-80 
전체 138 중 18, 주소:서울특별시 마포구 염리동 147-6 
전체 138 중 19, 주소:서울특별시 마포구 염리동 82-4번지 
전체 138 중 20, 주소:서울특별시 마포구 염리동 147-4번지 1층 
전체 138 중 21, 주소:서울특별시 마포구 염리동 148-9번지 
전체 138 중 22, 주소:서울특별시 마포구 염리동 9-120
전체 138 중 23, 주소:서울특별시 마포구 염리동 88-1
전체 138 중 24, 주소:서울특별시 마포구 염리동 168-40번지 
전체 138 중 25, 주소:서울특별시 마포구 염리동 147-5번지 2층 
전체 138 중 26, 주소:서울특별시 마포구 염리동 

In [7]:
naver_res

,지번주소,도로명주소,업체명,업종,URL,ID
0,서울특별시 마포구 염리동 148-13,"서울특별시 마포구 백범로 133 (염리동, 1층)",컴포즈커피 마포공덕점,카페,https://pcmap.place.naver.com/restaurant/15852...,1585262979
1,서울특별시 마포구 염리동 147-28,"서울특별시 마포구 숭문길 18, 1층 (염리동)",제주돈도야지,"육류,고기요리",https://pcmap.place.naver.com/restaurant/39261...,392617762
2,서울특별시 마포구 염리동 172-14,"서울특별시 마포구 독막로 310, 지상2층 (염리동)",태화루,중식당,https://pcmap.place.naver.com/restaurant/13553...,1355349231
3,서울특별시 마포구 염리동 172-14,"서울특별시 마포구 독막로 310, 지상2층 (염리동)",냉삼시대,"육류,고기요리",https://pcmap.place.naver.com/restaurant/12281...,1228100111
4,서울특별시 마포구 염리동 168-9 재화스퀘어,"서울특별시 마포구 독막로 311, 재화스퀘어 지하1층 B104호 (염리동)",무쇠뚜껑,"육류,고기요리",https://pcmap.place.naver.com/restaurant/11339...,1133945423
...,...,...,...,...,...,...
1,서울특별시 마포구 염리동 532 마포프레스티지자이,"서울특별시 마포구 대흥로24길 24, 근린생활시설-2(1획지)동 지1층 C-B2-1...",프랭크버거 마포프레스티지자이점,햄버거,https://pcmap.place.naver.com/restaurant/14163...,1416340049
2,서울특별시 마포구 염리동 532 마포프레스티지자이,"서울특별시 마포구 대흥로24길 24, 근린생활시설-2(1획지)동 지1층 C-B2-1...",에픽에스프레소,"카페,디저트",https://pcmap.place.naver.com/hairshop/1871901...,1871901008
0,서울특별시 마포구 염리동 157-14,"서울특별시 마포구 백범로24길 5-3, 1,2층 (염리동)",티엔플로우,베이커리,https://pcmap.place.naver.com/restaurant/11448...,1144898603
0,서울특별시 마포구 염리동 27-109,"서울특별시 마포구 마포대로11길 88, 1층 2호 일부호 (염리동)",난바우동,일식당,https://pcmap.place.naver.com/restaurant/14654...,1465458930


In [13]:
len(set(naver_res.ID)), len(set(naver_res.업체명))

(122, 125)

In [21]:
naver_res[naver_res['지번주소'] == '서울특별시 마포구 염리동 172-15 ']

,지번주소,도로명주소,업체명,업종,URL,ID
10,서울특별시 마포구 염리동 172-15,서울특별시 마포구 독막로 308 (염리동),마포낙지한마리수제비 마포점,낙지요리,https://pcmap.place.naver.com/restaurant/32049...,32049700
